# Black-Litterman Model: Generating views with ARMA & Monte-Carlo Simulation

In [75]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

# disable the convergence warning for poor ARIMA fit
import warnings
warnings.filterwarnings("ignore")

import empyrical as emp

## A. Loading data and train-test split

In [5]:
# load stocks
stock_names = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NFLX', 'NVDA', 'TSLA']
N = len(stock_names)
prices = pdr.get_data_yahoo(stock_names, start="2018-01-02", end="2020-01-04")['Adj Close']

prices_mat = np.asarray(prices)
X_log = np.diff(np.log(prices_mat), axis=0)
X_lin = np.diff(prices_mat, axis=0) / prices_mat[:-1]
T, N = X_log.shape

# train test split
no_train = int(T * 0.5)
X_log_train = X_log[:no_train,]
X_log_test = X_log[no_train:,]
X_lin_train = X_lin[:no_train,]
X_lin_test = X_lin[no_train:,]

# validation set for ARMA parameters from the train set
no_validate = int(no_train * 0.2)
X_log_validate_train = X_log_train[:no_train - no_validate]
X_log_validate = X_log_train[no_train - no_validate:]
X_lin_validate_train = X_lin_train[:no_train - no_validate]
X_log_validate = X_lin_train[no_train - no_validate:]

[*********************100%%**********************]  8 of 8 completed


## B. Determining optimal parameters for ARMA fit

In [6]:
optimal_arma_params = {}

# grid search for p in [0, 4] and q in [0, 4] for the best params based on the train and validation set
# judgement of performance is based on MSE
for i in range(N):
  stock = stock_names[i]

  # retrieve the train and validate returns for the stock
  validate_train = X_log_validate_train[:, i]
  validate = X_log_validate[:, i]

  best_p, best_q = 0, 0
  min_MSE = float('inf')

  for p in range(5):
    for q in range(5):
      fitted_model = ARIMA(validate_train, order=(p, 0, q), trend='c').fit()
      model_forecast = fitted_model.forecast(steps=len(validate))
      difference = np.array(validate - model_forecast)
      MSE = np.linalg.norm(difference)**2 / T

      # potentially update best_p, best_q, and min_MSE
      if MSE < min_MSE:
        min_MSE = MSE
        best_p = p
        best_q = q

    optimal_arma_params[stock] = (best_p, best_q)

optimal_arma_params

{'AAPL': (3, 4),
 'AMZN': (2, 4),
 'GOOGL': (2, 2),
 'META': (3, 2),
 'MSFT': (3, 3),
 'NFLX': (4, 4),
 'NVDA': (3, 3),
 'TSLA': (3, 4)}

## C. Fitting ARMA based on optimal parameters obtained

In [7]:
ARMA_forecast = {}

for i in range(N):
  stock = stock_names[i]
  opt_p, opt_q = optimal_arma_params[stock]
  full_train = X_log_train[:, i]

  # fit ARMA based on the optimal params
  # predict returns of 21-days ahead
  fitted_model = ARIMA(full_train, order=(opt_p, 0, opt_q), trend='c').fit()
  model_forecast = fitted_model.forecast(steps=21)[-1]
  ARMA_forecast[stock] = model_forecast

ARMA_forecast

{'AAPL': 0.0008902139728065171,
 'AMZN': 0.0009148877565231486,
 'GOOGL': 0.0017690296502717686,
 'META': -0.0012760136518959048,
 'MSFT': 0.004600820806724913,
 'NFLX': 0.001108848546311473,
 'NVDA': -0.00191109163987963,
 'TSLA': -0.0003772810494743478}

We only keep the tickers with non-negative predicted returns, and further proceed for Monte-Carlo Simulation for those stocks.

In [8]:
positive_stock_names = []

for key, val in ARMA_forecast.items():
  if val > 0: positive_stock_names.append(key)

positive_stock_names

['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'NFLX']

## D. Monte-Carlo Simulation for views generation

In [9]:
start_prices = {}
for stock in positive_stock_names:
  start_prices[stock] = prices.iloc[no_train][stock]

In [10]:
def GeometricBrownianMotion(T, nSteps, nPaths, X0 = 1 , mu = 1, sigma = 0.25):
  dt = T / nSteps
  X = np.zeros((nPaths, nSteps))
  X[:,0] = X0
  for i in range(nSteps-1):
    X[:,i+1] = X[:,i] * np.exp((mu -0.5*sigma**2)*dt + sigma*np.sqrt(dt)*np.random.randn(nPaths))
  return X

steps = 10000
paths = 10000
final_prices = {}  # record the final prices after 1 year

for stock in positive_stock_names:
  S0 = start_prices[stock]
  mu = 365 * np.log(np.exp(ARMA_forecast[stock]))
  stock_paths = GeometricBrownianMotion(1, steps, paths, S0, mu)
  final_prices[stock] = stock_paths[:, -1]

For the generated final prices, we only keep those that are greater than the start price, since we predicted a positive return.<br>
Next, we calculate the expected return and the corresponding variance.

In [18]:
positive_stock_mu = {}
positive_stock_var = {}

for stock in positive_stock_names:
  kept_prices = final_prices[stock][final_prices[stock] > start_prices[stock]]
  log_returns = np.log((final_prices[stock][final_prices[stock] > start_prices[stock]] / start_prices[stock]) ** (1 / 365))
  positive_stock_mu[stock] = log_returns.mean()
  positive_stock_var[stock] = log_returns.var()

## E. Black-Litterman Estimation

Based on the calculated mean and variance, we are ready to specify $\mathbf{P}$, $\mathbf{v}$, and $\boldsymbol{\Omega}$ in the Black-Litterman Model.<br>
Also, recall the formula for the Black-Litterman Estimators:<br>
$\boldsymbol{\mu_{BL}} = \mathbf{m} + \tau \boldsymbol{\Sigma} \mathbf{P}^T (\tau \mathbf{P} \boldsymbol{\Sigma} \mathbf{P}^T + \boldsymbol{\Omega})^{-1} (\mathbf{v} - \mathbf{P} \mathbf{m})$<br>
$\boldsymbol{\Sigma_{BL}} = (1 + \tau)\boldsymbol{\Sigma} - \tau^2 \boldsymbol{\Sigma} \mathbf{P}^T (\tau \mathbf{P} \boldsymbol{\Sigma} \mathbf{P}^T + \boldsymbol{\Omega})^{-1} \mathbf{P} \boldsymbol{\Sigma}$<br>

In [87]:
N_pos_stocks = len(positive_stock_names)
P = np.eye(N_pos_stocks)
v = np.array([positive_stock_mu[stock] for stock in positive_stock_names])
Omega = np.diag([positive_stock_var[stock] for stock in positive_stock_names])

# specify m and Sigma based on sample estimator
pos_stock_X_log_train = X_log_train[:, [stock_names.index(stock) for stock in positive_stock_names]]
m = np.mean(pos_stock_X_log_train, axis=0)
Sigma = np.cov(pos_stock_X_log_train.T)

tau = 1 / no_train

In [88]:
mu_BL = m + tau * Sigma @ P.T @ np.linalg.inv(tau * (P @ Sigma @ P.T) + Omega) @ (v - P @ m)
Sigma_BL = (1 + tau) * Sigma - (tau**2) * (Sigma @ P.T) @ np.linalg.inv(tau * (P @ Sigma @ P.T) + Omega) @ (P @ Sigma)

## F. Portfolio Optimization and comparison among portfolios

In [100]:
# max Sharpe portfolio
def compute_MSRP(mu, Sigma):
  w = cp.Variable(len(mu))
  variance = cp.quad_form(w, Sigma)
  problem = cp.Problem(cp.Minimize(variance), [w >= 0, w @ mu == 1])
  problem.solve()
  return w.value/np.sum(w.value)

# global min variance portfolio
def compute_GMVP(Sigma):
    w = cp.Variable(np.shape(Sigma)[0])
    variance = cp.quad_form(w, Sigma)
    problem = cp.Problem(cp.Minimize(variance),
                         [w >= 0, cp.sum(w) == 1])
    problem.solve()
    return w.value

# mean-CVaR portfolio
def compute_portfolioCVaR(X, lmd = 0.5, alpha = 0.95):
  T, N = np.shape(X)
  mu = np.mean(X, axis=0)
  w = cp.Variable(N)
  z = cp.Variable(T)
  zeta = cp.Variable(1)
  problem = cp.Problem(cp.Maximize(w @ mu - lmd*zeta - (lmd/(T*(1-alpha))) * cp.sum(z)),
                         [z >= 0, z >= - X @ w - zeta, w >= 0, cp.sum(w) == 1])
  problem.solve()
  return w.value

# mean Max-DD portfolio
def compute_portfolioMaxDD(X, c = 0.2):
  T, N = np.shape(X)
  X_cum = np.cumsum(X, axis=1)
  mu = np.mean(X, axis=0)
  # variables
  w = cp.Variable(N)
  u = cp.Variable(T)
  # problem
  prob = cp.Problem(cp.Maximize(w @ mu),
                    [w >= 0, cp.sum(w) == 1,
                     u <= X_cum @ w + c,
                     u >= X_cum @ w,
                     u[1:] >= u[:-1]])
  result = prob.solve()
  return w.value

In [101]:
weights_EWP = np.ones(N) / N
weights_MSRP = compute_MSRP(X_log_train.mean(axis=0), np.cov(X_log_train.T))
weights_GMVP = compute_GMVP(np.cov(X_log_train.T))
weights_CVAR = compute_portfolioCVaR(X_log_train)
weights_DD = compute_portfolioMaxDD(X_log_train, c = 0.24)
weights_BL = list(compute_MSRP(mu_BL, Sigma_BL))

# append zeros to the stocks no used in weights_BL
for i in range(N):
  stock = stock_names[i]
  if stock not in positive_stock_names: weights_BL.insert(i, 0)

In [102]:
portfolios = pd.DataFrame(np.column_stack([weights_EWP, weights_MSRP, weights_GMVP, weights_CVAR, weights_DD, weights_BL]),
                          columns=["EWP", "MSRP", "GMVP", "mean-CVaR", "mean-max DD", "BL"], index=stock_names)

In [104]:
ret_all = X_lin @ portfolios
ret_all_trn = ret_all.iloc[:no_train, ]
ret_all_tst = ret_all.iloc[no_train:, ]

# compute performance measures
res = pd.DataFrame({
    "Sharpe ratio (in-sample)":      ret_all_trn.apply(emp.sharpe_ratio).apply(lambda x: f"{x:.3}"),
    "Sharpe ratio (out-of-sample)":  ret_all_tst.apply(emp.sharpe_ratio).apply(lambda x: f"{x:.3}"),
    "Annual return (in-sample)":     ret_all_trn.apply(emp.annual_return).apply(lambda x: f"{x:.2%}"),
    "Annual return (out-of-sample)": ret_all_tst.apply(emp.annual_return).apply(lambda x: f"{x:.2%}"),
    "Annual volatility (in-sample)": ret_all_trn.apply(emp.annual_volatility).apply(lambda x: f"{x:.2%}"),
    "Annual volatility (out-of-sample)":
                                     ret_all_tst.apply(emp.annual_volatility).apply(lambda x: f"{x:.2%}"),
})
res

,Sharpe ratio (in-sample),Sharpe ratio (out-of-sample),Annual return (in-sample),Annual return (out-of-sample),Annual volatility (in-sample),Annual volatility (out-of-sample)
EWP,0.123,2.21,-0.78%,57.29%,29.90%,21.61%
MSRP,0.918,0.99,31.71%,24.07%,37.81%,24.90%
GMVP,-0.0296,2.75,-4.06%,66.30%,25.98%,19.18%
mean-CVaR,0.237,2.25,2.80%,52.37%,27.37%,19.59%
mean-max DD,0.381,2.26,7.26%,57.09%,30.76%,20.99%
BL,0.641,2.67,15.28%,65.26%,28.46%,19.56%
